In [103]:
import pandas as pd
import geopandas as gpd
import numpy as np
import os
import pathlib
import pyarrow.parquet as pq



In [100]:
# load a test data set got one pattern
sample_95 = gpd.read_parquet("../cta-stop-etl/out/trips/test_trips_95_small.parquet")
sample_3910 = gpd.read_parquet("../cta-stop-etl/out/trips/test_trips_3910_small.parquet")

In [101]:
sample_95_stops = sample_95[sample_95['typ']== 'S']
sample_3910_stops = sample_3910[sample_3910['typ']== 'S']
sample = pd.concat([sample_95_stops,sample_3910_stops])
sample.head()


,seg_combined,typ,bus_stop_time,time_diff,time_diff_seconds,stop_dist,speed_mph,unique_trip_vehicle_day,stpid,p_stp_id,geometry,data_time,b_value,ping_time_diff
0,0.0,S,2023-01-04 23:36:45.000000000,NaT,15.900749,207.698618,29.219337,7295.0235314066107561419402023-01-04,923,95-923,POINT (358249.87291 582489.58505),NaT,NaN,NaT
7,4.0,S,2023-01-04 23:47:32.408154452,19361 days 23:47:32.408154452,15.582072,203.535996,29.219337,7295.0235314066107561419402023-01-04,924,95-924,POINT (358177.75661 582481.33075),NaT,NaN,NaT
8,9.0,S,2023-01-04 23:47:55.400228857,0 days 00:00:22.992074405,22.987198,300.263161,29.219337,7295.0235314066107561419402023-01-04,926,95-926,POINT (357979.93959 582483.38174),NaT,NaN,NaT
9,16.0,S,2023-01-04 23:48:55.847421941,0 days 00:01:00.447193084,60.434372,251.905079,9.324107,7295.0235314066107561419402023-01-04,927,95-927,POINT (357690.92237 582477.25608),NaT,NaN,NaT
10,21.0,S,2023-01-04 23:49:46.136671140,0 days 00:00:50.289249199,50.278583,266.080393,11.838159,7295.0235314066107561419402023-01-04,928,95-928,POINT (357449.61839 582470.41645),NaT,NaN,NaT


In [102]:
sample['bus_stop_date'] = pd.to_datetime(sample['bus_stop_time'].dt.date)
grouping = sample.groupby(['p_stp_id', 'bus_stop_date']).size().reset_index(name = 'count')
grouping.groupby('p_stp_id')['count'].mean().reset_index(name = 'bus_per_day').head(100)

,p_stp_id,bus_per_day
0,3910-1061,1.234568
1,3910-1062,1.234568
2,3910-1163,1.234568
3,3910-12549,1.234568
4,3910-12550,1.234568
...,...,...
95,95-845,20.000000
96,95-923,20.000000
97,95-924,20.000000
98,95-926,20.000000


In [ ]:
def buses_per_stop():
    """
    calculate avg bus per bus stop by day
    """
    #load all pids
    pids = pd.read_csv('../cta-stop-etl/out/stops/all_pids_list.parquet')

    all_data = []
    for pid in pids:
        pid = int(pid)
        # returns list of dict for each row
        data = pq.read_table(f'../cta-stop-etl/out/trips/trips_{pid}.parquet').to_pylist()
        all_data += data
        print(f'pid: {pid} loading done')

    full_df = pd.DataFrame(data)

    full_df['bus_stop_date'] = pd.to_datetime(full_df['bus_stop_time'].dt.date)
    grouping = sample.groupby(['p_stp_id', 'bus_stop_date']).size().reset_index(name = 'count')
    final_df = grouping.groupby('p_stp_id')['count'].mean().reset_index(name = 'bus_per_day')

    return final_df